In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingModelConfig:
    root_dir : Path
    trained_model_path : Path
    updated_base_model_path : Path
    training_data : Path
    params_epochs : int
    params_batch_size : int
    params_is_augmentation : bool
    params_img_size : list

@dataclass(frozen=True)
class PrepareCallBacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path


In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [9]:
from pathlib import Path

project_root = Path(r"C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project")

CONFIG_FILE_PATH = project_root / "config" / "config.yaml"
PARAMS_FILE_PATH = project_root / "params.yaml"

print("Config file path:", CONFIG_FILE_PATH)
print("Params file path:", PARAMS_FILE_PATH)

print("Config exists?", CONFIG_FILE_PATH.is_file())
print("Params exists?", PARAMS_FILE_PATH.is_file())


Config file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
Params file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
Config exists? True
Params exists? True


In [49]:
class ConfigurationManager:
    def __init__ (self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root], verbose=True)

    def get_prepare_callbacks_config (self) -> PrepareCallBacksConfig : 
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)],verbose = True)
            
        prepare_callback_config = PrepareCallBacksConfig(
            root_dir= Path(config.root_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir)
        )
            
        return prepare_callback_config 
    
    def get_training_config(self) -> TrainingModelConfig:
        training = self.config.training
        prepare_base_model_config = self.get_prepare_base_model_config()
        params = self.params

        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([training.root_dir], verbose=True)

        training_config = TrainingModelConfig(
        root_dir=training.root_dir,
        trained_model_path=training.trained_model_path,
        updated_base_model_path=prepare_base_model_config.updated_base_model_path,
        training_data=training_data,
        params_epochs=params.EPOCHS,
        params_batch_size=params.BATCH_SIZE,
        params_is_augmentation=params.AUGMENTATION,
        params_img_size=params.IMAGE_SIZE
    )

        return training_config



In [15]:
import time
import tensorflow as tf

In [16]:
class PrepareCallbacks:
    def __init__ (self, config:PrepareCallBacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ] 

In [ ]:
from pathlib import Path
import os
import urllib.request as request
from zipfile import ZipFile
from cnnClassifier import logger
import time
import tensorflow as tf
tf.config.run_functions_eagerly(True)  
import tensorflow as tf
tf.config.run_functions_eagerly(True) 




In [47]:
tf.config.run_functions_eagerly(True)
print("Eager execution:", tf.executing_eagerly())
class Training :
    def __init__(self, config : TrainingModelConfig):
        self.config = config
        

    # load the model
    def get_base_model(self):
        self.model= tf.keras.models.load_model(self.config.updated_base_model_path)
        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

    def train_valid_generator(self):
        # create a data generator for training and validation data
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.20
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_img_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        # reads the image and picks only the validation data
        valid_datageneartor = tf.keras.preprocessing.image.ImageDataGenerator(** datagenerator_kwargs)

        self.valid_generator = valid_datageneartor.flow_from_directory(
            directory = self.config.training_data,
            subset= "validation",
            shuffle = False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation :
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range = 40,
                horizontal_flip = True,
                width_shift_range = 0.2,
                height_shift_range = 0.2,
                shear_range = 0.2,
                zoom_range=0.2,
                ** datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datageneartor

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path : Path , model : tf.keras.Model):
        model.save(path)

    def train(self, callbacks_list: list):
        self.steps_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        #training the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_steps=self.validation_steps,
            validation_data = self.valid_generator,
            callbacks = callbacks_list
            
        )

        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )






    

Eager execution: True


In [48]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callbacks_list= callback_list
    )
    
except Exception as e:
    raise e



✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
✅ DEBUG YAML CONTENT: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/PrachiVedant/datasets/raw/refs/heads/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': ' artifacts/training/model.keras'}} <class 'dict'>
✅ DEBUG YAML PATH: C

[2025-07-08 12:16:21,559 : WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.5493 - loss: 8.0554[2025-07-08 12:17:30,640 : WARNING : saving_api : You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')` or `keras.saving.save_model(model, 'my_model.keras')`. ]
10/10 ━━━━━━━━━━━━━━━━━━━━ 68s 7s/step - accuracy: 0.5480 - loss: 8.3321 - val_accuracy: 0.3906 - val_loss: 20.3451


In [36]:
import tensorflow as tf
print(tf.__version__)



2.19.0
